In [1]:
!git clone https://github.com/Bhavnoor-Coders-1010/food101.git

Cloning into 'food101'...
remote: Enumerating objects: 6220, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 6220 (delta 1), reused 16 (delta 1), pack-reused 6200 (from 4)
Receiving objects: 100% (6220/6220), 169.23 MiB | 14.95 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (6256/6256), done.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.optimizers import Adam
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [3]:
#defining labels
classes={0 : 'burger',
1 : 'butter_naan',
2 : 'chai',
3 : 'chapati',
4 : 'chole_bhature',
5 : 'dal_makhani',
6 : 'dhokla',
7 : 'fried_rice',
8 : 'idli',
9 : 'jalebi',
10 : 'kathi_roll',
11 : 'kadhai_paneer',
12 : 'kulfi',
13 : 'masala_dosa',
14 : 'momos',
15 : 'paani_puri',
16 : 'pakode',
17 : 'pav_bhaji',
18 : 'pizza',
19 : 'samosa'
           }

In [4]:
def print_mapped_labels(label_indices, label_dic):
    mapped_labels = [label_dic[index] for index in label_indices]
    print(mapped_labels)

In [5]:
train_df = pd.read_csv('/content/food101/DataFOOD101/train.csv')
test_df = pd.read_csv('/content/food101/DataFOOD101/test.csv')
train_df.head()

,imagePath,label
0,0342.jpg,2
1,1077.jpg,0
2,2314.jpg,11
3,3068.jpg,0
4,4121.jpg,13


In [7]:
#extracting image with help of file location given in csv and resizig it
def load_and_preprocess_image(image_path, image_folder,target_size=(224, 224)):
    try:
        full_path = os.path.join(image_folder, image_path)
        image = load_img(full_path, target_size=target_size)
        image = img_to_array(image)
        image = image / 255.0 #normalizing the array
        return image
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return None

In [9]:
#applying data augmentation on file
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [10]:
train_images = []
train_labels = []
for img_path, label in zip(train_df['imagePath'], train_df['label']):
    image_train = load_and_preprocess_image(img_path,'/content/food101/DataFOOD101/Train')
    if image_train is not None:
        train_images.append(image_train)
        train_labels.append(label)

In [11]:
train_images = np.array(train_images)
train_labels = to_categorical(train_labels, num_classes=20)

In [12]:
test_images = []
test_labels = []
for img_path, label in zip(test_df['imagePath'], test_df['label']):
    image_test = load_and_preprocess_image(img_path,'/content/food101/DataFOOD101/Test')
    if image_test is not None:
        test_images.append(image_test)
        test_labels.append(label)

In [13]:
test_images = np.array(test_images)
test_labels = to_categorical(test_labels, num_classes=20)

In [14]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# making last 84 layers trainable for fine-tuning of MobileNet model
for layer in base_model.layers[-84:]:
    layer.trainable = True

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
print('Number of images in the training dataset:', train_images.shape[0])
print('Number of images in the testing dataset:', test_images.shape[0])

Number of images in the training dataset: 5314
Number of images in the testing dataset: 939


In [17]:
print(f"Shape of the images in the training dataset: {train_images[0].shape}")

Shape of the images in the training dataset: (224, 224, 3)


In [18]:
model = Sequential([
    base_model,
      GlobalAveragePooling2D(),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(256, activation='relu'),
      Dropout(0.5),
      Dense(20, activation='softmax')
])

In [19]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(datagen.flow(train_images, train_labels, batch_size=32), epochs=20, validation_data=(test_images, test_labels))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


167/167 ━━━━━━━━━━━━━━━━━━━━ 122s 512ms/step - accuracy: 0.2717 - loss: 2.6810 - val_accuracy: 0.5091 - val_loss: 2.0796
Epoch 2/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 101s 395ms/step - accuracy: 0.6333 - loss: 1.3880 - val_accuracy: 0.5836 - val_loss: 1.7229
Epoch 3/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 68s 391ms/step - accuracy: 0.7142 - loss: 1.1163 - val_accuracy: 0.6230 - val_loss: 1.4757
Epoch 4/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 71s 410ms/step - accuracy: 0.7500 - loss: 0.9697 - val_accuracy: 0.5740 - val_loss: 1.8204
Epoch 5/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 79s 392ms/step - accuracy: 0.7756 - loss: 0.8625 - val_accuracy: 0.6763 - val_loss: 1.2779
Epoch 6/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 83s 396ms/step - accuracy: 0.7773 - loss: 0.8414 - val_accuracy: 0.7359 - val_loss: 1.0301
Epoch 7/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 68s 393ms/step - accuracy: 0.8023 - loss: 0.7487 - val_accuracy: 0.6731 - val_loss: 1.2011
Epoch 8/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 66s 381ms/step - accuracy: 0.8142 - loss: 0.7092 - v

In [25]:
model.save('food_classifier_model.h5')

In [27]:
from tensorflow.keras.models import load_model


loaded_model = load_model('food_classifier_model.h5')

In [28]:
import cv2
def predict(image_path):
    food_labels = [
    "burger",
    "butter_naan",
    "chai",
    "chapati",
    "chole_bhature",
    "dal_makhani",
    "dhokla",
    "fried_rice",
    "idli",
    "jalebi",
    "kathi_roll",
    "kadhai_paneer",
    "kulfi",
    "masala_dosa",
    "momos",
    "paani_puri",
    "pakode",
    "pav_bhaji",
    "pizza",
    "samosa"
    ]
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = loaded_model.predict(img)
    predicted_class_index = np.argmax(prediction)
    predicted_food_item = food_labels[predicted_class_index]

    return predicted_food_item
